In [2]:
import numpy as np
import pandas as pd
from geopy.distance import geodesic
from itertools import combinations
import random
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)


In [3]:
# Load city data and calculate the distance matrix
def load_cities_and_distance_matrix(file_path):
    cities = pd.read_csv(file_path, header=None, names=['name', 'lat', 'lon'])
    num_cities = len(cities)
    
    dist_matrix = np.zeros((num_cities, num_cities))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = dist
    
    return cities, dist_matrix

# Calculate the total cost of the TSP path
def tsp_cost(tsp, dist_matrix):
    if tsp[0] != tsp[-1]:
        logging.error("Path must start and end at the same city.")
        return float('inf')  # Return a large number if the assertion fails
    if set(tsp) != set(range(len(dist_matrix))):
        logging.error("Path must include all cities.")
        return float('inf')  # Return a large number if the assertion fails

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost


# FIRST AND FASTER APPROACH: GREEDY ALGORITHM (NEAREST NEIGHBOUR)

In [4]:
# Greedy Algorithm (Nearest Neighbor)
def nearest_neighbor(cities, dist_matrix):
    num_cities = len(cities)
    visited = np.full(num_cities, False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = [city]

    while not np.all(visited):
        dist[:, city] = np.inf  # Mark current city as visited
        closest = np.argmin(dist[city])
        logging.debug(
            f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f} km)"
        )
        visited[closest] = True
        city = closest
        tsp.append(city)

    logging.debug(
        f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f} km)"
    )
    tsp.append(tsp[0])  # Return to starting city

    logging.info(f"Greedy result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp, dist_matrix):.2f} km")
    return tsp


# SECOND SLOWER BUT MORE ACCURATE APPROACH

Steady-State Genetic Algorithm (GA) for TSP Optimization


In [5]:
# Generate an initial population of random paths
def generate_initial_population(size, num_cities):
    return [np.random.permutation(num_cities).tolist() + [0] for _ in range(size)]  # Ensure it returns to start city

# Cycle Crossover
def cycle_crossover(parent1, parent2):
    size = len(parent1) - 1  # Exclude the last city (return to start)
    child1 = [-1] * size
    child2 = [-1] * size
    
    # Find cycles
    visited = [False] * size
    for start in range(size):
        if not visited[start]:
            current = start
            while not visited[current]:
                child1[current] = parent1[current]
                visited[current] = True
                current = parent2.index(parent1[current])
    
    # Fill in the remaining cities
    for i in range(size):
        if child1[i] == -1:
            for gene in parent2:
                if gene not in child1:
                    child1[i] = gene
                    break

    # Create second child using the same logic
    visited = [False] * size
    for start in range(size):
        if not visited[start]:
            current = start
            while not visited[current]:
                child2[current] = parent2[current]
                visited[current] = True
                current = parent1.index(parent2[current])
    
    for i in range(size):
        if child2[i] == -1:
            for gene in parent1:
                if gene not in child2:
                    child2[i] = gene
                    break

    # Ensure both children return to the starting city
    return child1 + [child1[0]], child2 + [child2[0]]

# Insert Mutation
def insert_mutation(path):
    new_path = path[:-1]  # Exclude return to start city
    i, j = random.sample(range(1, len(new_path)), 2)  # Select two indices
    # Insert gene at j into position i
    gene = new_path.pop(j)
    new_path.insert(i, gene)
    return new_path + [new_path[0]]  # Ensure it returns to start city

# Steady-State Evolutionary Algorithm
def steady_state_evolutionary_algorithm(cities, dist_matrix, population_size=10, generations=50):
    population = generate_initial_population(population_size, len(cities))
    
    for generation in range(generations):
        # Select parents
        parents = random.sample(population, 2)
        
        # Generate children using cycle crossover
        child1, child2 = cycle_crossover(parents[0], parents[1])
        
        # Mutate children
        if random.random() < 0.2:  # Mutation probability
            child1 = insert_mutation(child1)
        if random.random() < 0.2:  # Mutation probability
            child2 = insert_mutation(child2)
        
        # Replace the worst members of the population
        population.extend([child1, child2])
        population = sorted(population, key=lambda x: tsp_cost(x, dist_matrix))[:population_size]  # Keep best individuals
    
    # Find the best path in the final population
    best_path = min(population, key=lambda x: tsp_cost(x, dist_matrix))
    return best_path


In [8]:
# Main function to solve TSP for the cities file
def solve_tsp(file_path):
    # Load cities and distance matrix
    cities, dist_matrix = load_cities_and_distance_matrix(file_path)
    
    # Run Greedy Algorithm
    greedy_path = nearest_neighbor(cities, dist_matrix)
    greedy_cost = tsp_cost(greedy_path, dist_matrix)
    
    logging.info(f"Greedy path: {greedy_path}, Cost: {greedy_cost:.2f} km")
    
    # Run Steady-State Evolutionary Algorithm
    genetic_path = steady_state_evolutionary_algorithm(cities, dist_matrix)
    genetic_cost = tsp_cost(genetic_path, dist_matrix)
    
    logging.info(f"Genetic path: {genetic_path}, Cost: {genetic_cost:.2f} km")

# Run TSP solutions on a single file "italy.csv"
if __name__ == "__main__":
    solve_tsp("/Users/nikolastankovic/Downloads/italy.csv")
    #solve_tsp("/Users/nikolastankovic/Downloads/china.csv")
    #solve_tsp("/Users/nikolastankovic/Downloads/russia.csv")
    #solve_tsp("/Users/nikolastankovic/Downloads/us.csv")
    #solve_tsp("/Users/nikolastankovic/Downloads/vanuatu.csv")


DEBUG:root:step: Ancona -> Rimini (90.60 km)
DEBUG:root:step: Rimini -> Forlì (46.72 km)
DEBUG:root:step: Forlì -> Ravenna (26.46 km)
DEBUG:root:step: Ravenna -> Ferrara (66.67 km)
DEBUG:root:step: Ferrara -> Bologna (43.43 km)
DEBUG:root:step: Bologna -> Modena (37.29 km)
DEBUG:root:step: Modena -> Reggio nell'Emilia (23.94 km)
DEBUG:root:step: Reggio nell'Emilia -> Parma (26.94 km)
DEBUG:root:step: Parma -> Piacenza (57.65 km)
DEBUG:root:step: Piacenza -> Milan (60.65 km)
DEBUG:root:step: Milan -> Monza (14.51 km)
DEBUG:root:step: Monza -> Bergamo (33.92 km)
DEBUG:root:step: Bergamo -> Brescia (46.02 km)
DEBUG:root:step: Brescia -> Verona (61.42 km)
DEBUG:root:step: Verona -> Vicenza (44.70 km)
DEBUG:root:step: Vicenza -> Padua (30.13 km)
DEBUG:root:step: Padua -> Venice (36.07 km)
DEBUG:root:step: Venice -> Trieste (115.09 km)
DEBUG:root:step: Trieste -> Bolzano (209.68 km)
DEBUG:root:step: Bolzano -> Trento (49.94 km)
DEBUG:root:step: Trento -> Novara (206.69 km)
DEBUG:root:step: N